In [2]:
import numpy as np
import matplotlib.pyplot as plt
from matplotlib import gridspec
import seaborn as sns
import os, re
import glob
import _pickle as pkl
import scipy.signal as signal
import math
import pandas as pd
%matplotlib inline
%load_ext autoreload
%autoreload 2 
sns.set_style("white")
import warnings
warnings.filterwarnings('ignore')

from dlab import generalephys as ephys
import dlab as traces
import dlab as utils

In [3]:
def count_all_peaks(file_path, sigma_thresh=5):
    bad_channels = []
    traces.skip_channels = np.append(traces.npix_p3_reference_channels, bad_channels)
    
    mm = np.memmap(file_path, dtype=np.int16, mode='r')
    
    num_channels = traces.get_channel_count(file_path.rsplit(os.path.sep, 1)[0], from_channel_map=False)
    chunk = traces.get_chunk(mm, 3., 11., num_channels, sampling_rate = 30000)
    
    norm_chunk, mean_offset = get_norm_chunk(chunk)
    
    peaks = np.zeros((norm_chunk.shape[0] - len(traces.skip_channels), 1))
    dead_channels_passed = 0
    
    for channel in range(norm_chunk.shape[0]):
        if channel not in traces.skip_channels:
            peaks[channel - dead_channels_passed], _, _ = count_channel_peaks(norm_chunk[channel,:], sigma_thresh)
        else:
            dead_channels_passed += 1
        
    return peaks, mean_offset

def count_channel_peaks(signal, sigma_thresh=5):
    
    num_peaks = 0
    indices = []
    peak_values = []
    threshold = sigma_thresh * np.std(signal) + np.mean(signal)
    
    above_thresh = False
    max_point = 0
    
    for index, point in enumerate(signal):
        if above_thresh:
            if point < threshold * 0.5:
                indices.append(peak_index)
                peak_values.append(max_point)
                max_point = 0
                above_thresh = False
            else: 
                if point > max_point:
                    max_point = point
                    peak_index = index
                
        else:
            if point >= threshold:
                above_thresh = True
                num_peaks = num_peaks + 1
                peak_index = index
                max_point = point
                
    return num_peaks, indices, peak_values

def get_norm_chunk(chunk):
    chunk_detrended = np.zeros(chunk.shape)
    mean_offset = np.zeros((chunk.shape[0], 1))
    
    for channel in range(chunk.shape[0]):
        chunk_detrended[channel, :] = butter_highpass_filter(chunk[channel,:], 10, 30000, order=5)
        mean_offset[channel, :] = np.mean(chunk[channel,:])
    
    return chunk_detrended, mean_offset

def butter_highpass(cutoff, fs, order=5):
    nyq = 0.5 * fs
    normal_cutoff = cutoff / nyq
    b, a = signal.bessel(order, normal_cutoff, btype='highpass', analog=False)
    w, h = signal.freqs(b, a)
    return b, a

def butter_highpass_filter(data, cutoff, fs, order=5):
    b, a = butter_highpass(cutoff, fs, order=order)
    y = signal.filtfilt(b, a, data)
    return y

def running_mean(x, N):
    return np.expand_dims(np.convolve(x[:,0], np.ones((N,))/N)[(N-1) / 2:-(N-1) / 2], 1)

## Choose Files for Data

In [6]:
path = os.path.join(r'\Users\danieljdenman','Desktop')

# filenames_spikeband = glob.glob(os.path.join(path,'*','*0_0.dat'))
# filenames_lfp = glob.glob(os.path.join(path,'*','*1_0.dat'))

# phase = 3
# bad_channels = []
# traces.skip_channels = np.append(traces.npix_p3_reference_channels, bad_channels)

In [60]:
print traces.skip_channels

[  36.   75.  112.  151.  188.  227.  264.  303.  340.  379.]


# Download this dataset to your local machine start:
localization/M310016_2017-06-15_09-11-15_3
### this can be done through denmanlab.quickconnect.to

## Get Labels

In [15]:
labels.keys()

dict_keys([b'/Volumes/SD1/DanD/M310016/localization/M310016_2017-06-15_09-11-15_3', b'/Volumes/SD1/DanD/M310016/localization/M310016_2017-06-15_08-43-17_2', b'/Volumes/SD1/DanD/M310016/localization/M310016_2017-06-15_09-55-55_4b', b'/Volumes/SD1/DanD/M310016/localization/M310016_2017-06-15_11-21-18_7', b'/Volumes/SD1/DanD/M310016/localization/M310016_2017-06-15_11-51-37_8', b'/Volumes/SD1/DanD/M310016/localization/M310016_2017-06-15_09-44-15_4', b'/Volumes/SD1/DanD/M310016/localization/M310016_2017-06-15_10-25-04_5', b'/Volumes/SD1/DanD/M310016/localization/M310016_2017-06-15_08-10-38_1', '/Volumes/SD1/DanD/M270512/270512_2016-12-16_10-51-44'])

In [27]:
labels_path = 'labels.pkl'
labels = pkl.load(open(labels_path,'rb'))

## This just takes the first dictionary keys and shortens them down a little to just something like 
## M310016_2017-06-15_08-10-38_1 instead of the whole filename
# for key in labels.keys():
#     print(type(str(key)))#.split('/')
#     labels[str(key).split('/')[-1]]# = labels.pop(key)

In [29]:
labels

{b'/Volumes/SD1/DanD/M310016/localization/M310016_2017-06-15_10-52-16_6': {'ventral thalamus': (0,
   39),
  'primary somatosensory cortex': (279, 351),
  'white matter': (40, 47),
  'above': (352, 383),
  'lateral geniculate': (48, 88),
  'CA3': (89, 148),
  'CA1': (190, 268),
  'dentate gyrus': (149, 189)},
 b'/Volumes/SD1/DanD/M310016/localization/M310016_2017-06-15_09-11-15_3': {'secondary motor cortex': (184,
   331),
  'lateral ventricle': (44, 160),
  'white matter': (161, 183),
  'above': (332, 383),
  'fimbria': (0, 43)},
 b'/Volumes/SD1/DanD/M310016/localization/M310016_2017-06-15_08-43-17_2': {'caudate putamen': (72,
   185),
  'primary motor cortex': (215, 345),
  'white matter': (186, 214),
  'above': (346, 383),
  'globus pallidus': (0, 71)},
 b'/Volumes/SD1/DanD/M310016/localization/M310016_2017-06-15_09-55-55_4b': {'primary somatosensory cortex': (268,
   339),
  'lateral ventricle': (147, 243),
  'white matter': (244, 267),
  'above': (340, 383),
  'VPM': (0, 37),
  'L

In [32]:
## Display all the annotations for a quick reference when I need to look at something
for i, dirname in enumerate(filenames_lfp):
    dirkey = dirname.split(os.path.sep)[-2]
    print(dirkey)
    for structure in labels[dirkey].iterkeys():
        print('             ', structure, labels[dirkey][structure][0], labels[dirkey][structure][1])

NameError: name 'filenames_lfp' is not defined

## GETTING DATA (NOT CHUNKED)

Meaning for each channel individually. Chunking it comes later on.

In [43]:
target_all_ = np.empty((0,1))
rms_spike_all_ = np.empty((0,1))
rms_lfp_all_ = np.empty((0,1))
gamma_all_ = np.empty((0,1))
alpha_all_ = np.empty((0,1))
beta_all_ = np.empty((0,1))
delta_all_ = np.empty((0,1))
theta_all_ = np.empty((0,1))
peaks_all_ = np.empty((0,1))
mean_offset_all_ = np.empty((0,1))
row_all_ = np.empty((0,1))

for i in range(len(filenames_lfp)):
    spike_file = filenames_spikeband[i]
    lfp_file = filenames_lfp[i]
    dirkey = spike_file.split(os.path.sep)[-2]
    print dirkey, i
    
    rms_spike_ = np.expand_dims(traces.get_probe_freq(spike_file, frequency_range=[300, 30000]), 1)
    rms_lfp_ = np.expand_dims(traces.get_probe_freq(lfp_file, frequency_range=[0, 300]), 1)
    gamma_ = np.expand_dims(traces.get_probe_freq(lfp_file, frequency_range=[30, 40]), 1)
    alpha_ = np.expand_dims(traces.get_probe_freq(lfp_file, frequency_range=[8, 12]), 1)
    beta_ = np.expand_dims(traces.get_probe_freq(lfp_file, frequency_range=[12, 30]), 1)
    delta_ = np.expand_dims(traces.get_probe_freq(lfp_file, frequency_range=[0, 4]), 1)
    theta_ = np.expand_dims(traces.get_probe_freq(lfp_file, frequency_range=[4, 8]), 1)
    peaks_, mean_offset_ = count_all_peaks(spike_file, 4)
    peaks_ = running_mean(peaks_, 10)
    row_ = np.expand_dims([pix // 2 for pix in range(384)], 1)
    
    target_ = ["" for j in range(384)]
    
    for structure in labels[dirkey].iterkeys():
        for index in np.arange(labels[dirkey][structure][0], labels[dirkey][structure][1] + 1):
            target_[index] = structure
    
    for skip in sorted(traces.skip_channels, reverse=True):
        target_.pop(int(skip))
        row_ = np.delete(row_, int(skip), 0)
        mean_offset_ = np.delete(mean_offset_, int(skip), 0)
    
    ## 82nd channel is always weird and can't add it to bad channels because then a different channel becomes weird, and so on
    rms_spike_[82,:] = (rms_spike_[81,:] + rms_spike_[83,:]) / 2.0
    rms_lfp_[82,:] = (rms_lfp_[81,:] + rms_lfp_[83,:]) / 2.0
    gamma_[82,:] = (gamma_[81,:] + gamma_[83,:]) / 2.0
    alpha_[82,:] = (alpha_[81,:] + alpha_[83,:]) / 2.0
    beta_[82,:] = (beta_[81,:] + beta_[83,:]) / 2.0
    delta_[82,:] = (delta_[81,:] + delta_[83,:]) / 2.0
    theta_[82,:] = (theta_[81,:] + theta_[83,:]) / 2.0
    
    rms_spike_all_ = np.append(rms_spike_all_, rms_spike_, axis=0)
    rms_lfp_all_ = np.append(rms_lfp_all_, rms_lfp_, axis=0)
    gamma_all_ = np.append(gamma_all_, gamma_, axis=0)
    alpha_all_ = np.append(alpha_all_, alpha_, axis=0)
    beta_all_ = np.append(beta_all_, beta_, axis=0)
    delta_all_ = np.append(delta_all_, delta_, axis=0)
    theta_all_ = np.append(theta_all_, theta_, axis=0)
    peaks_all_ = np.append(peaks_all_, peaks_, axis=0)
    mean_offset_all_ = np.append(mean_offset_all_, mean_offset_, axis=0)
    row_all_ = np.append(row_all_, row_, axis=0)
    target_all_ = np.append(target_all_, np.expand_dims(target_, 1), axis=0)

all_bands_data = np.concatenate((rms_spike_all_, rms_lfp_all_, gamma_all_, alpha_all_, 
                                 beta_all_, delta_all_, theta_all_, peaks_all_, 
                                 mean_offset_all_, row_all_), axis=1)

## fixes the double labeling of white matter... kinda hacky though (where there were two dict entries for white matter)
for index in np.where(target_all_ == '')[0]:
    target_all_[index] = 'white matter'

M310016_2017-06-15_08-10-38_1 0
M310016_2017-06-15_08-43-17_2 1
M310016_2017-06-15_09-11-15_3 2
M310016_2017-06-15_09-44-15_4 3
M310016_2017-06-15_09-55-55_4b 4
M310016_2017-06-15_10-25-04_5 5
M310016_2017-06-15_10-52-16_6 6
M310016_2017-06-15_11-21-18_7 7
M310016_2017-06-15_11-51-37_8 8


In [44]:
print all_bands_data.shape

(3366L, 10L)
